In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

In [8]:
#Load CSV
df = pd.read_csv("../mergedata_%change.csv")
df.head()

,Unnamed: 0,SPX Ti,Date Tf,SPX Tf,Source,Tweet,Date Ti,Retweets,Favorites,SA,% Change SPX
0,0,2244.00,20170102 1849,2244.25,Twitter for Android,I thought and felt I would win big easily over...,20170102 1844,13437,62670,1,0.011141
1,1,2242.75,20170103 1235,2243.25,Twitter for Android,General Motors is sending Mexican made model o...,20170103 1230,18864,73104,1,0.022294
2,2,2240.75,20170103 1251,2241.00,Twitter for Android,People must remember that ObamaCare just doesn...,20170103 1246,14481,58809,-1,0.011157
3,3,2241.00,20170103 1256,2242.75,Twitter for Android,"The Democrat Governor.of Minnesota said ""The A...",20170103 1251,12599,53372,-1,0.078090
4,4,2245.25,20170103 1508,2245.50,Twitter for Android,With all that Congress has to work on do they ...,20170103 1503,10851,43062,-1,0.011135


In [34]:
#Bin SPX % Change into +1,0,-1 and establish threshold
market_movements = []
pos_threshold = 0.0001
neg_threshold = pos_threshold * -1

for index, row in df.iterrows():
    if row['% Change SPX'] >= pos_threshold:
        market_movement = 1
        market_movements.append(market_movement)
    elif row['% Change SPX'] <= neg_threshold:
        market_movement = -1
        market_movements.append(market_movement)
    else:
        market_movement = 0
        market_movements.append(market_movement)

#Add new list as column to df
df['Market Movement'] = market_movements
df

,Unnamed: 0,SPX Ti,Date Tf,SPX Tf,Source,Tweet,Date Ti,Retweets,Favorites,SA,% Change SPX,Market Movement
0,0,2244.000,20170102 1849,2244.25,Twitter for Android,I thought and felt I would win big easily over...,20170102 1844,13437,62670,1,0.011141,1
1,1,2242.750,20170103 1235,2243.25,Twitter for Android,General Motors is sending Mexican made model o...,20170103 1230,18864,73104,1,0.022294,1
2,2,2240.750,20170103 1251,2241.00,Twitter for Android,People must remember that ObamaCare just doesn...,20170103 1246,14481,58809,-1,0.011157,1
3,3,2241.000,20170103 1256,2242.75,Twitter for Android,"The Democrat Governor.of Minnesota said ""The A...",20170103 1251,12599,53372,-1,0.078090,1
4,4,2245.250,20170103 1508,2245.50,Twitter for Android,With all that Congress has to work on do they ...,20170103 1503,10851,43062,-1,0.011135,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2950,2950,2498.470,20181228 1247,2497.97,Twitter for iPhone,....The United States looses soooo much money ...,20181228 1242,23736,101103,1,-0.020012,-1
2951,2951,2495.070,20181228 1254,2496.47,Twitter for iPhone,.....close the Southern Border. Bring our car ...,20181228 1249,25309,108158,1,0.056111,1
2952,2952,2489.669,20181228 1311,2493.67,Twitter for iPhone,.....Honduras Guatemala and El Salvador are do...,20181228 1306,39548,167132,-1,0.160704,1
2953,2953,2516.669,20181228 1456,2517.17,Twitter for iPhone,Thank you to Sean Parnell for the nice comment...,20181228 1451,21407,113861,1,0.019907,1


In [35]:
#State target output, name the %change bins
target = df["Market Movement"]
target_names = ["Positive", "No Effect", "Negative"]

In [36]:
#Remove Output column to avoid data leakage
data = df.drop("Market Movement", axis=1)
feature_names = data.columns
data.head()

,Unnamed: 0,SPX Ti,Date Tf,SPX Tf,Source,Tweet,Date Ti,Retweets,Favorites,SA,% Change SPX
0,0,2244.00,20170102 1849,2244.25,Twitter for Android,I thought and felt I would win big easily over...,20170102 1844,13437,62670,1,0.011141
1,1,2242.75,20170103 1235,2243.25,Twitter for Android,General Motors is sending Mexican made model o...,20170103 1230,18864,73104,1,0.022294
2,2,2240.75,20170103 1251,2241.00,Twitter for Android,People must remember that ObamaCare just doesn...,20170103 1246,14481,58809,-1,0.011157
3,3,2241.00,20170103 1256,2242.75,Twitter for Android,"The Democrat Governor.of Minnesota said ""The A...",20170103 1251,12599,53372,-1,0.078090
4,4,2245.25,20170103 1508,2245.50,Twitter for Android,With all that Congress has to work on do they ...,20170103 1503,10851,43062,-1,0.011135


In [37]:
# tokenize, then remove stopwords, ngrams, sentiment analysis feature selection
#either truncate or append to vector

## Count Vectorizer

In [60]:
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer

corpus = df['Tweet']

vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1, 3), stop_words = 'english')
vector = vectorizer2.fit_transform(corpus)
vector

<2955x80131 sparse matrix of type '<class 'numpy.int64'>'
	with 128391 stored elements in Compressed Sparse Row format>

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector.toarray(), target, random_state=42)

In [62]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [63]:
print('Test Acc: %.4f' % model.score(X_test, y_test))

# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

Test Acc: 0.4208
              precision    recall  f1-score   support

    Positive       0.41      0.46      0.43       310
   No Effect       0.31      0.05      0.08       102
    Negative       0.44      0.50      0.47       327

    accuracy                           0.42       739
   macro avg       0.39      0.34      0.33       739
weighted avg       0.41      0.42      0.40       739



## Hashing Vectorizer

In [44]:
from sklearn.feature_extraction.text import HashingVectorizer

# list of text documents
text = data["Tweet"]
# create the transform
vectorizer = HashingVectorizer(n_features=2000, ngram_range=(1, 3))
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(2955, 2000)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.08006408  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.10540926  0.          0.         ...  0.          0.
   0.        ]]


In [45]:
vector

<2955x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 226631 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector.toarray(), target, random_state=42)

In [47]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [48]:
print('Test Acc: %.4f' % model.score(X_test, y_test))

Test Acc: 0.4073


In [49]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

    Positive       0.41      0.48      0.44       310
   No Effect       0.00      0.00      0.00       102
    Negative       0.41      0.47      0.44       327

    accuracy                           0.41       739
   macro avg       0.27      0.32      0.29       739
weighted avg       0.35      0.41      0.38       739



C:\Users\Natha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
target = data["RTs"]
target_names = ["Positive", "No Effect", "Negative"]

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector.toarray(), target, random_state=42)

In [49]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [50]:
print('Test Acc: %.4f' % model.score(X_test, y_test))

Test Acc: 0.0200
